In [21]:
import pandas as pd
import numpy as np
import duckdb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [22]:
def preprocess_imdb_data(data_path, directors_path, writers_path):
    """
    General preprocessing pipeline for IMDB data.
    
    Arguments:
    - data_path: Path to the train/test/validation data CSV file.
    - directors_path: Path to the directing.json file.
    - writers_path: Path to the writing.json file.
    
    Returns:
    - Cleaned Pandas DataFrame ready for model training or prediction.
    """
    
    # Step 1: Load main dataset
    df = pd.read_csv(data_path)

    # Step 2: Load JSON files (Directors & Writers)
    df_directors = pd.read_json(directors_path)
    df_writers = pd.read_json(writers_path)

    # Step 3: Rename columns for consistency
    df_directors.rename(columns={"movie": "tconst", "director": "director_id"}, inplace=True)
    df_writers.rename(columns={"movie": "tconst", "writer": "writer_id"}, inplace=True)

    # Step 4: Convert nested JSON fields into strings (fixes 'unhashable type' error)
    df_directors["director_id"] = df_directors["director_id"].astype(str)
    df_writers["writer_id"] = df_writers["writer_id"].astype(str)

    # Step 5: Merge main dataset with Directors & Writers using DuckDB
    con = duckdb.connect()
    con.register("movies", df)
    con.register("directors", df_directors)
    con.register("writers", df_writers)

    query = """
    SELECT 
        movies.*, 
        directors.director_id, 
        writers.writer_id
    FROM movies
    LEFT JOIN directors ON movies.tconst = directors.tconst
    LEFT JOIN writers ON movies.tconst = writers.tconst
    """

    df = con.execute(query).fetchdf()
    con.close()

    # Step 6: Drop unnecessary columns
    columns_to_drop = ["originalTitle", "endYear", "Unnamed: 0"]
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    # Step 7: Handle missing values
    numeric_columns = ["startYear", "runtimeMinutes", "numVotes"]
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")  # Ensure numeric format
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())  # Fill missing with median

    df["director_id"] = df["director_id"].fillna("unknown")
    df["writer_id"] = df["writer_id"].fillna("unknown")

    # Step 8: Ensure correct data types
    df["startYear"] = df["startYear"].astype(int)
    df["runtimeMinutes"] = df["runtimeMinutes"].astype(int)
    df["numVotes"] = df["numVotes"].astype(int)

    # Step 9: Ensure each `tconst` is unique
    df = df.groupby("tconst").first().reset_index()
    
    return df


In [23]:
# Define file paths
train_files = [
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-1.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-2.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-3.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-4.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-5.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-6.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-7.csv",
    "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/train-8.csv"
]

directors_path = "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/directing.json"
writers_path = "C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/writing.json"

# Preprocess and merge all training data
df_train = pd.concat([preprocess_imdb_data(file, directors_path, writers_path) for file in train_files], ignore_index=True)

# Preprocess validation and test data
df_val = preprocess_imdb_data("C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/validation_hidden.csv", directors_path, writers_path)
df_test = preprocess_imdb_data("C:/Users/Gebruiker/Documents/UVA/Vakken met code/BD/BigData-Group10/imdb/test_hidden.csv", directors_path, writers_path)

# Save cleaned datasets
df_train.to_csv("final_training_data.csv", index=False)
df_val.to_csv("final_validation_data.csv", index=False)
df_test.to_csv("final_test_data.csv", index=False)

print("\n✅ All datasets have been preprocessed and saved!")



✅ All datasets have been preprocessed and saved!


In [24]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define Features & Target
features = ["startYear", "runtimeMinutes", "numVotes", "director_id", "writer_id"]
X_train = df_train[features]
y_train = df_train["label"]
X_val = df_val[features]
X_test = df_test[features]

# Preprocessing Pipeline
numeric_features = ["startYear", "runtimeMinutes", "numVotes"]
categorical_features = ["director_id", "writer_id"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

# Train Logistic Regression Model
model = Pipeline([
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

print("🔹 Training model on full training data...")
model.fit(X_train, y_train)

# Generate Predictions
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Save predictions in required format (no headers, single column)
pd.DataFrame(y_val_pred).to_csv("validation_predictions.csv", index=False, header=False)
pd.DataFrame(y_test_pred).to_csv("test_predictions.csv", index=False, header=False)

print("✅ Predictions saved for submission!")


🔹 Training model on full training data...
✅ Predictions saved for submission!
